# Résolution Pb65 de la collection Hock-Schittkowski

Classification dans l'ouvrage : QQR-P1-3

Number of variables : n = 3

Number of constraints : 7 (1 inequality + 6 bounds)

# Modèle

$\left\{  
\begin{array}{lllll} 
\min f(x) \\ 
\text{s.c.}\\
 48-x_1^2-x_2^2-x_3^2\geq 0\\
-4.5\leq x_i\leq 4.5,\text{ pour } i=1,2\\
-5 \leq x_3 \leq 5
\end{array} \right.$

où $f : x \longmapsto (x_1-x_2)^2 + \dfrac{(x_1+x_2-10)^2}{9}+(x_3-5)^2=||r(x)||^2$

avec $r : x\longmapsto  (x_1-x_2,\dfrac{x_1-x_2-10}{3},x_3-5)$

Point de départ : $x_0 = (-5,5,0)$ (non réalisable)

Fonction objectif en $x_0$ : $\dfrac{1225}{9}$

Solution attendue : $x^* = (3.650461821,3.65046168,4.6204170507)$

Fonction objectif à la solution : $f(x^*) = 0.935288567$

In [1]:
using LinearAlgebra, Polynomials, Printf, JuMP, Ipopt, BenchmarkTools

In [2]:
path_enlsip_file = string(pwd(),"/enlsip-julia-0.2.0.jl")
include(path_enlsip_file)

In [3]:
n = 3
m = 3
nb_eq = 0
nb_constraints = 7

7

In [4]:
res65 = ResidualsEval(0)

function (res65::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(res65.ctrl) == 1
        rx[:] = [x[1] - x[2]; (x[1]+x[2]-10.0) / 3.0; x[3]-5.0]

    # The jacobian is computed analytically
    elseif res65.ctrl == 2
        J[:] = [1. -1. 0;
                1/3 1/3 0.;
                0. 0. 1.]
    end
    return
end

In [5]:
cons65 = ConstraintsEval(0)

function (cons65::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(cons65.ctrl) == 1
        cx[:] = [48.0 - x[1]^2-x[2]^2-x[3]^2;
                 x[1]+4.5;
                 x[2]+4.5;
                 x[3]+5.0;
                 -x[1]+4.5;
                 -x[2]+4.5;
                 -x[3]+5.0]
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif cons65.ctrl == 2
        cons65.ctrl = 0
    end
    return
end

In [6]:
x0 = [-5.0;5.0;0.0]
enlsip_020(x0,res65,cons65,n,m,nb_eq,nb_constraints)

****************************************
*                                      *
*          ENLSIP-JULIA-0.2.0          *
*                                      *
****************************************

Starting point : [-5.0, 5.0, 0.0]

Number of equality constraints   : 0
Number of inequality constraints : 7

iter    objective    cx_sum   method   ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  1.158907e+02  4.50e+00     1   5.002e+00   2     1   4.81e-01    0.00e+00     1.00e-01    (1,2,6)
   1  7.831910e+01  4.66e+01     1   6.835e+00   2     1   4.30e-01    1.80e+00     1.00e-01    (1,6)
   2  4.665009e+00  6.61e-02     1   8.193e+00   2     1   7.58e-01    8.91e-01     1.00e-01    (6,7)
   3  9.547961e-01  2.18e+01     1   1.439e+00   2     1   1.00e+00    5.82e-01     1.00e-01    (1,7)
   4  9.376705e-01  4.29e+00     1   4.655e-01   1     2   1.00e+00    4.18e-01     1.00e-01    (1)
   5  9.532018e-01  4.70e-02     1   6.668e-02   1     2   1.00e+0

In [7]:
x1_sol = enlsip_020.sol[1]
x2_sol = enlsip_020.sol[2]
x3_sol = enlsip_020.sol[3]

println("Solution trouvée avec ENLSIP-Julia :")
@printf "x_jul = (%.9e, %.9e, %.9e)\n" x1_sol x2_sol x3_sol
@printf "f(x_jul) = %.9e\n\n" enlsip_020.obj_value  

println("Solution trouvée avec ENLSIP-Fortran :")
@printf "x_for = (%.7e, %.7e, %.7e)\n"  3.6504617  3.6504617  4.6204176
@printf "f(x_for) = %.5e\n"  0.953529

println("\nValeurs théoriques visées :")
@printf "x_opt = (%.9e, %.9e, %.9e)\n" 3.650461821 3.65046168 4.6204170507
@printf "f(x_opt) = %.9e" 0.9535288567

Solution trouvée avec ENLSIP-Julia :
x_jul = (3.650461729e+00, 3.650461728e+00, 4.620417551e+00)
f(x_jul) = 9.535288568e-01

Solution trouvée avec ENLSIP-Fortran :
x_for = (3.6504617e+00, 3.6504617e+00, 4.6204176e+00)
f(x_for) = 9.53529e-01

Valeurs théoriques visées :
x_opt = (3.650461821e+00, 3.650461680e+00, 4.620417051e+00)
f(x_opt) = 9.535288567e-01

# Résolution avec Ipopt

In [8]:
model = Model(with_optimizer(Ipopt.Optimizer))

@variable(model, x1, start = -5.0)
@variable(model, x2, start = 5.0)
@variable(model, x3, start = 0.0)

x3

In [9]:
f(x1,x2,x3) = (x1 - x2)^2 + (x1+x2-10.0)^2 / 9.0 + (x3-5.0)^2
JuMP.register(model, :f, 3, f, autodiff=true)

In [10]:
@NLconstraint(model, c1, 48.0 - x1^2 - x2^2 - x3^2 >= 0)
@constraint(model, x1 >= -4.5)
@constraint(model, -x1 <= 4.5)
@constraint(model, x2 >= -4.5)
@constraint(model, x2 <= 4.5)
@constraint(model, x3 >= -5.0)
@constraint(model, x3 <= 5.0)

x3 ≤ 5.0

In [11]:
@NLobjective(model, Min, f(x1,x2,x3))

In [12]:
JuMP.optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        9
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [13]:
println("Solution trouvée avec ENLSIP :")
@printf "x_enlsip = (%.9e, %.9e, %.9e)\n" x1_sol x2_sol x3_sol
@printf "f(x_enlsip) = %.9e\n\n" enlsip_020.obj_value 


println("Solution trouvée avec Ipopt :")
@printf "x_ipopt = (%.9e, %.9e, %.9e)\n" JuMP.value(x1) JuMP.value(x2) JuMP.value(x3)
@printf "f(x_ipopt) = %.9e\n" 0.95352885599324944

Solution trouvée avec ENLSIP :
x_enlsip = (3.650461729e+00, 3.650461728e+00, 4.620417551e+00)
f(x_enlsip) = 9.535288568e-01

Solution trouvée avec Ipopt :
x_ipopt = (3.650461727e+00, 3.650461725e+00, 4.620417555e+00)
f(x_ipopt) = 9.535288560e-01
